In [1]:
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import numpy as np

# Retrieval with TF-IDF and BM25

## Loading the data

In [2]:
documents = [
    "Tech Stocks Rally as Investors Bet on Strong Quarterly Earnings",
    "Global Markets Dip Amid Concerns Over Slowing Tech Sector Growth",
    "Cryptocurrency Prices Surge Following Regulatory Clarity in Europe",
    "Federal Reserve Hints at Future Rate Cuts Boosting Market Confidence",
    "Oil Prices Fall as Supply Chain Disruptions Ease Worldwide",
    "Major Bank Reports Record Profits Driven by Consumer Lending",
    "Retail Stocks Drop After Weak Holiday Sales Forecast",
    "Automakers Invest Heavily in Electric Vehicles to Stay Competitive",
    "Investors Pull Back from Risky Assets Amid Inflation Fears",
    "Fintech Startups Gain Momentum with New Digital Payment Solutions",
]

## Stop Word Removal and Stemming

### Stop Word Removal
Stop words are common words in a language (like ```"the"```, ```"is"```, ```"and"```) that carry little semantic meaning. Removing them reduces noise and often improves the efficiency of text processing.

**Example:**

Original: ```"The cat is sitting on the mat."```
After Stop Word Removal: ```"cat sitting mat"```

### Stemming
Stemming reduces words to their root form, so that different variants of a word are treated as the same token. This helps in matching similar concepts.

**Example:**

Words: ```"running", "runs", "ran"```
After Stemming: ```"run"```

In [3]:
# Initialize stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(TfidfVectorizer(stop_words='english').get_stop_words())


### Task 1:

Complete the preprocessing function in the code chunk below.

The input of the function is a string of text.
It returns a list of stemmed, non-stop words.

1. Split the text into single words. (Hint: use the [```split()```](https://docs.python.org/3/library/stdtypes.html#str.split) function)
2. For each word word check if it is a stop word using the ```stop_words``` set.
3. Finally stem all the remaining words using [```stemmer.stem()```](https://www.nltk.org/api/nltk.stem.porter.html#nltk.stem.porter.PorterStemmer.stem).
4. Return the resulting list.

In [4]:

def preprocess(text: str) -> list[str]:
    """
    Preprocess the input text by tokenizing, removing stopwords, and stemming.
    
    Args:
        text (str): The input text to preprocess.
    
    Returns:
        list[str]: A list of preprocessed tokens.
    """
    # Your code here:
    tokens = text.lower().split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return tokens

In [5]:
# Preprocess documents
processed_docs = [' '.join(preprocess(doc)) for doc in documents]

## TF-IDF: Term Frequency–Inverse Document Frequency

**TF-IDF** is a numerical statistic that reflects how important a word is to a document within a collection (corpus).
It is widely used in information retrieval and text mining to rank documents by relevance.

**1. Term Frequency (TF)**

Measures how often a term appears in a document.

$$
\text{TF}(t, d) = \frac{\text{Number of times term } t \text{ appears in } d}{\text{Total number of terms in } d}
$$

**2. Inverse Document Frequency (IDF)**

Measures how informative a term is — rare terms get a higher score.

$$
\text{IDF}(t) = \log\left(\frac{N}{1 + n_t}\right)
$$

Where:

* $N$ = total number of documents
* $n_t$ = number of documents containing term *t*

(The $+1$ prevents division by zero.)

**3. TF-IDF Score**

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)
$$

**Interpretation**

* **High TF-IDF** → The term appears frequently in the document but rarely elsewhere → *important*.
* **Low TF-IDF** → The term is common across documents or rare in the current one → *less informative*.

## Using TF-IDF Vectorization

Now it is time to apply the TF-IDF vectorization to our preprocessed data.
Fortunately we don't have to code this ourselfs.
The vectorization is implemented for example in [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

We vectorize the query the same way as the documents and calculate the TF-IDF vectorization.
Then the cosine similarity can be used to find the best matching results.

In [6]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_docs)

In [ ]:
query = "tech sector earnings"
# query = "energy supply price drop"
# query = "automobile sector investments"
preprocessed_query = ' '.join(preprocess(query))
query_vector = tfidf_vectorizer.transform([preprocessed_query])

# Compute cosine similarity
cosine_sim = (tfidf_matrix @ query_vector.T).toarray().flatten()
tfidf_ranking = np.argsort(-cosine_sim)

print("TF-IDF Ranking:")
for idx in tfidf_ranking:
    print(f"Score: {cosine_sim[idx]:.4f} | Doc: {documents[idx]}")

TF-IDF Ranking:
Score: 0.2673 | Doc: Automakers Invest Heavily in Electric Vehicles to Stay Competitive
Score: 0.2474 | Doc: Global Markets Dip Amid Concerns Over Slowing Tech Sector Growth
Score: 0.0000 | Doc: Tech Stocks Rally as Investors Bet on Strong Quarterly Earnings
Score: 0.0000 | Doc: Cryptocurrency Prices Surge Following Regulatory Clarity in Europe
Score: 0.0000 | Doc: Federal Reserve Hints at Future Rate Cuts Boosting Market Confidence
Score: 0.0000 | Doc: Oil Prices Fall as Supply Chain Disruptions Ease Worldwide
Score: 0.0000 | Doc: Major Bank Reports Record Profits Driven by Consumer Lending
Score: 0.0000 | Doc: Retail Stocks Drop After Weak Holiday Sales Forecast
Score: 0.0000 | Doc: Investors Pull Back from Risky Assets Amid Inflation Fears
Score: 0.0000 | Doc: Fintech Startups Gain Momentum with New Digital Payment Solutions


### Task 2:

In the code above, experiment with different queries.
Can you find weaknesses with TF-IDF vectorization?

### Issues with TF-IDF

- **Bag-of-words assumption (no semantics)**  
  TF–IDF ignores word order, syntax and meaning. Synonyms and paraphrases (e.g., "car" vs "automobile") won’t match, and semantically similar documents can be missed.

- **Polysemy and ambiguity**  
  A single token can have multiple senses; TF–IDF treats them identically and can return irrelevant documents.

- **Sparse, high-dimensional vectors**  
  Representations are large and sparse, which can be memory- and compute-inefficient at scale, and sensitive to vocabulary mismatch and OOV tokens.

- **Length and frequency bias**  
  Raw term frequency and document length can bias scores (longer docs more likely to contain query terms). IDF can overemphasize rare noise words if corpus statistics are unstable.

- **No contextual or phrase understanding**  
  Multi-word expressions and context-dependent meaning are poorly handled; phrase matches are often missed unless explicitly indexed.

- **Preprocessing dependency**  
  Stemming, lemmatization, stopword lists, tokenization, and case handling can dramatically change results; mismatches in preprocessing between query and documents cause retrieval failures.

- **Scaling and sparsity in multilingual/morphologically rich languages**  
  Languages with rich morphology, diacritics or compound words pose additional challenges for lexical-only matching.

Mitigations include BM25 (better length normalization), query expansion, relevance feedback, LSI/LSA or SVD to capture latent topics, and modern dense (embedding-based) retrieval or hybrid sparse+dense approaches for semantic matching.